In [17]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, StratifiedKFold

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

iris = load_iris()

In [ ]:
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['label'] = iris.target
iris_df['label'].value_counts()

In [ ]:
kfold = KFold(n_splits=3)
# 여기 n_splits가 몇 중첩인가 이거구만
# 단 n_splits >= 2 ::: 0이나 1이면 굳이 중첩시킬 필요가 있어?

n_iter = 0
# 이 매개변수 그냥 반복문 i 같은 놈인 것 같은데 반복문 안에 못 집어넣을까?

for train_index, test_index in kfold.split(iris_df):
    n_iter += 1

    # loc vs. iloc 차이는 파이썬 기초부분이니 복습을 하자
    # 쉽게 말하면 데이터프레임에 loc는 라벨로 접근하는 거고 iloc는 인덱스로 접근한다는 것이다
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('## 교차검증 : {0}'.format(n_iter))
    print('학습 레이블 데이터 분포 : \n', label_train.value_counts())
    print('검증 레이블 데이터 분포: \n', label_test.value_counts())

In [21]:
# 위에 거를 바탕으로 예측까지 가봐?

kfold = KFold(n_splits=3)
n_iter = 0
dt_clf = DecisionTreeClassifier(random_state=121)
var_list = []

for train_index, test_index in kfold.split(iris.data):
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]

    X_train, X_test = iris.data[train_index], iris.data[test_index]
    y_train, y_test = iris.target[train_index], iris.target[test_index]

    dt_clf.fit(X_train, y_train)

    print('## 교차검증 : {0}'.format(n_iter))
    print('학습 레이블 데이터 분포 : \n', label_train.value_counts())
    print('검증 레이블 데이터 분포: \n', label_test.value_counts())

    pred = dt_clf.predict(X_test)
    print("예측: ", pred)

    accuracy = np.round(accuracy_score(y_test, pred),4)
    print(accuracy)
    var_list.append(accuracy)

## 교차검증 : 1
학습 레이블 데이터 분포 : 
 1    50
2    50
Name: label, dtype: int64
검증 레이블 데이터 분포: 
 0    50
Name: label, dtype: int64
예측:  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]
0.0
## 교차검증 : 2
학습 레이블 데이터 분포 : 
 0    50
2    50
Name: label, dtype: int64
검증 레이블 데이터 분포: 
 1    50
Name: label, dtype: int64
예측:  [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 0 2]
0.0
## 교차검증 : 3
학습 레이블 데이터 분포 : 
 0    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포: 
 2    50
Name: label, dtype: int64
예측:  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]
0.0


* 근데 예측 결과 보면 알겠지만 개떡이야. 이게 예측한 게 맞나 싶을 정도로 문제임. 특히 예측률
* 분명히 분류대상은 0,1,2 인 걸 iris 써봤으니 아는데 이거... 제대로 학습한 거 맞아?
* 이건 단순히 데이터셋에서 학습용 데이터셋을 나눠가지고 fold를 돌리는 게 만사가 아니다
* 기계한테 큰 그림을 그릴 수 있도록 뭔가 조치를 취해줘야할 것만 같다
* 그래서 단순 fold의 단점을 극복하고자 straified K-Fold 라는 걸 써볼겁니다

# StraifiedKFold
* https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn.model_selection.StratifiedKFold
* sklearn.model_selection.StratifiedKFold(n_splits=5, *, shuffle=False, random_state=None)

In [27]:
sk_f = StratifiedKFold(n_splits=3)
n_iter = 0
dt_clf = DecisionTreeClassifier(random_state=121)
var_list = []

# split 할 때 data, target 둘 중 하나 없으면 에러 뱉을거임.
for train_index, test_index in sk_f.split(iris.data, iris.target):
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]

    X_train, X_test = iris.data[train_index], iris.data[test_index]
    y_train, y_test = iris.target[train_index], iris.target[test_index]

    print('## 교차검증 : {0}'.format(n_iter))
    print('학습 레이블 데이터 분포 : \n', label_train.value_counts())
    print('검증 레이블 데이터 분포: \n', label_test.value_counts())
    
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    print("예측: ", pred)

    accuracy = np.round(accuracy_score(y_test, pred),4)
    print(accuracy)
    var_list.append(accuracy)

## 교차검증 : 1
학습 레이블 데이터 분포 : 
 2    34
0    33
1    33
Name: label, dtype: int64
검증 레이블 데이터 분포: 
 0    17
1    17
2    16
Name: label, dtype: int64
예측:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2
 2 2 2 1 2 2 2 2 2 2 2 2 2]
0.98
## 교차검증 : 2
학습 레이블 데이터 분포 : 
 1    34
0    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포: 
 0    17
2    17
1    16
Name: label, dtype: int64
예측:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 2 1 1 1 1 1 1 2 1 1 1 1 1 2 2 2 1
 2 2 2 2 2 2 2 2 2 2 2 2 2]
0.94
## 교차검증 : 3
학습 레이블 데이터 분포 : 
 0    34
1    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포: 
 1    17
2    17
0    16
Name: label, dtype: int64
예측:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2]
0.98


* 근데 이거 매번 for문 돌려야 하는 거야? 더 간결한 거 없어?

# sklearn.model_selection cross_val_score()
* https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html
* https://guru.tistory.com/35

sklearn.model_selection.cross_val_score(estimator, X, y=None, *, groups=None, scoring=None, cv=None, n_jobs=None, verbose=0, fit_params=None, pre_dispatch='2*n_jobs', error_score=nan)

* 이제와서 적어보는 수식 보는 tip : =None 같이 값이 있는 것들은 공식문서 상에 default 값들이 있어서 써도 되고 안 써도 되는 건데
* 이런 거 없이 그냥 있는 애들은 값을 반드시 입력해야만 한다는 얘기임
* 하이퍼 파라미터들 중 핵심은 estimator, X, cv(=fold 개수) : 공식문서와 구글링을 통해 파악을 해봅시다

In [28]:
from sklearn.model_selection import cross_val_score, cross_validate

In [31]:
# 이제 복잡하게 for문 쓰지 말고 간단하게 교차검증을 해보도록 합시다

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state = 156)

data = iris_data.data
label = iris_data.target

# 성능 지표는 정확도(accuracy), 교차 검증 세트는 3개
scores = cross_val_score(dt_clf, data, label, scoring="accuracy", cv=3)

print('교차 검증별 정확도: ', np.round(scores,4))
print('평균 검증 정확도: ', np.round(np.mean(scores),4))

# 너무 간단해서 눈물이 다 나오네?

교차 검증별 정확도:  [0.98 0.94 0.98]
평균 검증 정확도:  0.9667
